# 文本预处理

In [1]:
#导入第三方库
%matplotlib inline
import collections, re
from d2l import torch as d2l

In [2]:
#读取时光机器数据集
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine():
    """将时间机器数据集读取到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        #读取文本行
        lines = f.readlines()
    #使用正则表达式保留26个字母，其余使用空格代替
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
#打印信息
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [3]:
#词元化(tokenize)
#将文本序列拆分成词元列表并返回列表，每个词元是字符串的形式
def tokenize(lines, token= 'word'):
    """将文本拆分成单词或字符词元"""
    #单词形式进行拆分
    if token == 'word':
        return [line.split() for line in lines]
    #字符形式直接转变为列表
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型' + token)

tokens = tokenize(lines)
for i in range(15):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
['fire', 'burned', 'brightly', 'and', 'the', 'soft', 'radiance', 'of', 'the', 'incandescent']
['lights', 'in', 'the', 'lilies', 'of', 'silver', 'caught', 'the', 'bubbles', 'that', 'flashed', 'and']
['passed', 'in', 'our', 'glasses', 'our', 'chairs', 'being', 'his', 'patents', 'embraced', 'and']
['caressed', 'us', 'rather', 'than', 'submitted', 'to', 'be', 'sat', 'upon', 'and', 'there', 'was', 'that']


In [4]:
#构建一个字典(也称词表)，用于将字符串类型的词元映射到从0开始的数字索引中
class Vocab:
    """构建文本词表"""
    def __init__(self, tokens= None, min_freq= 0, reserved_tokens= None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        #按词频排序
        counter = count_corpus(tokens)
        #按词频大到小展示
        self._token_freqs = sorted(counter.items(), key= lambda x: x[1],
                                   reverse= True)
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
        
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]        
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    #未知词元索引设置为0
    def unk(self):
        return 0

    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    """统计词元频率"""
    #tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        #将词元展平成一个列表(嵌套循环)
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [6]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [7]:
#将每一个文本行转换为1个数字索引列表
for i in [0, 10]:
    print('文本：', tokens[i])
    print('索引：', vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引： [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [8]:
#整合所有功能
def load_corpus_time_machine(max_tokens= -1):
    """返回时光机器数据集的词元索引和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)